# Week 2  - Part 3
## Segmenting and Clustering Neighbourhoods in Toronto
### Foursquare API: Exploring Neighbourhoods of Cities in Toronto
---

Activities:

* Explore and cluster the neighbourhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

* Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
- to generate maps to visualize your neighbourhoods and how they cluster together. 
- Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

---

In [176]:
#Importing Libraries

# Data handling
import numpy as np 

# Data Analysis
import pandas as pd 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# JSON files
import json 

# Address converted in latitude and longitude
from geopy.geocoders import Nominatim 

#  JSON file converted to Pandas DataFrame
from pandas.io.json import json_normalize 

# Matplotlib and Plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

#Requests
import requests

print('Success!')

Success!


In [177]:
# Loading last DataFrame Version df_P2, from Part 2
df = pd.read_csv('df_P2.csv')
print(df.shape)
df.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [178]:
# Exploring & Clustering Neighbourhoods in Toronto

address = 'Toronto, ON'

#UserAgent: http request sent with request
geolocator = Nominatim(user_agent="agent") 
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('Coordinates of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

Coordinates of Toronto are 43.6534817, -79.3839347.


In [179]:
# Creating a Toronto Map
Toronto_map=folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)
#Toronto_map

#Selecting only Borough containing "Toronto"
df= df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(df.shape)
df.head()   


(39, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306


In [180]:
# Adding Markers and Styling to the Map
for lat, lng, borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='pink',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
#Toronto_map

In [181]:
#Foursquare API Connection
#ID
CLIENT_ID = ''
#PWR
CLIENT_SECRET = ''
VERSION = '20200611'

print('Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Credentials:
CLIENT_ID: GSRJY4CTWD1XFQHOIW5HBHXFJ3DUQ3EJ04NZIL3XWQ22WC35
CLIENT_SECRET:FYRBOSNJBFWRGUKBZDM3BCI31VLKRZPLACL2ZRVEVCITDMAJ


## Toronto Neighbourhood Exploration

In [182]:
#Listing All Nearby Venues
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API request URL Creation
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Selecting all relevant information about Nearby Venues
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [183]:
# Retrieving Toronto Venues information about Cordinates
toronto_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [184]:
# Printing Neighbourhood and Venue Cordinates Data (Latitude and Longitude) with Venue Categories
print(toronto_venues.shape)
toronto_venues.head()

(1601, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65512,-79.36264,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
3,"Regent Park, Harbourfront",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space
4,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


In [185]:
#Grouping Toronto Venues & Counting
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,48,48,48,48,48,48
"Brockton, Parkdale Village, Exhibition Place",83,83,83,83,83,83
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",77,77,77,77,77,77
Central Bay Street,53,53,53,53,53,53
Christie,9,9,9,9,9,9
Church and Wellesley,67,67,67,67,67,67
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,21,21,21,21,21,21
Davisville North,8,8,8,8,8,8


In [186]:
# Categories Quantity
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


In [187]:
# One-hot-encoding: transform Venue Category variables in binary variables
toronto_ohe = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#toronto_ohe['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_ohe.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_ohe.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )

#fixed_columns = [toronto_ohe.columns[-1]] + list(toronto_ohe.columns[:-1])
#toronto_ohe = toronto_ohe[fixed_columns]

toronto_ohe.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cable Car,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [188]:
#Shaping One Hot Encoding from Venue Category
toronto_ohe.shape

(1601, 214)

In [189]:
#Grouping According to Neighbourhood
toronto_group = toronto_ohe.groupby('Neighborhood').mean().reset_index()
toronto_group.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cable Car,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.0,0.0,0.0,0.020833,0.0,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.062500,0.020833,0.000000,0.0,0.041667,0.0,0.0,0.020833,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.020833,0.0,0.020833,0.0,0.0,0.0,0.020833,0.000000,0.0,0.000000,0.000000,0.083333,0.062500,0.0,0.000000,0.0,0.0,0.020833,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.041667,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.020833,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.020833,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020833,0.000000,0.0,0.000000,0.020833,0.000000,0.000000,0.020833,0.020833,0.020833,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.020833,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.

In [190]:
#Shaping DataFrame Grouped by Neighbourhood after One Hot Encoding thorough Venue Category
toronto_group.shape

(38, 214)

In [191]:
#Returning Most Commom Venues according to each row (Neighbourhood)
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Defining Variables to Return Commom Venues
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Populating DataFrame
neigh_venues_sorted = pd.DataFrame(columns=columns)
neigh_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_group.shape[0]):
   neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neigh_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Butcher,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Coffee Shop,Sandwich Place,Restaurant,Japanese Restaurant,Café,Yoga Studio,Pizza Place,Lounge,Italian Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Sandwich Place,Bar,Park,Intersection,French Restaurant,Spa,Speakeasy,Restaurant
3,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Pizza Place,Sushi Restaurant,Plaza,Middle Eastern Restaurant,Japanese Restaurant,Bank,Cosmetics Shop
4,Christie,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio


## Neighbourhood Clustering Process

In [192]:
clusters = 5
toronto_group_cluster = toronto_group.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=clusters, random_state=1).fit(toronto_group_cluster)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[0 0 0 0 0 0 0 0 0 0]
38


In [193]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306


In [194]:
neigh_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Butcher,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Coffee Shop,Sandwich Place,Restaurant,Japanese Restaurant,Café,Yoga Studio,Pizza Place,Lounge,Italian Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Sandwich Place,Bar,Park,Intersection,French Restaurant,Spa,Speakeasy,Restaurant
3,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Pizza Place,Sushi Restaurant,Plaza,Middle Eastern Restaurant,Japanese Restaurant,Bank,Cosmetics Shop
4,Christie,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio


In [195]:
# add clustering labels
neigh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#df = df.join(neigh_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
df=df.join(neigh_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

df.head()
#neigh_venues_sorted.head()
neigh_venues_sorted.to_csv('neigh.csv',index=False)

In [196]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0.0,Coffee Shop,Restaurant,Distribution Center,Bakery,Wine Shop,Event Space,Breakfast Spot,Discount Store,Greek Restaurant,Sushi Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0.0,Coffee Shop,Sandwich Place,Hotel,Café,Clothing Store,Cosmetics Shop,Pizza Place,Japanese Restaurant,Middle Eastern Restaurant,Bank
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,0.0,Coffee Shop,Italian Restaurant,Cocktail Bar,Restaurant,Gastropub,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Moroccan Restaurant
3,M4E,East Toronto,The Beaches,43.67709,-79.29547,0.0,Health Food Store,Playground,Pub,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,0.0,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Butcher,Restaurant


## Final Clusters

In [197]:
clusters_map = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

x = np.arange(clusters)
ys = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=7, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(clusters_map)
clusters_map

## Cluster 1 - Restaurant Route

In [198]:
df.loc[df['Cluster Labels'] == 0, df.columns[[1] + list(range(5, df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Coffee Shop,Restaurant,Distribution Center,Bakery,Wine Shop,Event Space,Breakfast Spot,Discount Store,Greek Restaurant,Sushi Restaurant
1,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,Hotel,Café,Clothing Store,Cosmetics Shop,Pizza Place,Japanese Restaurant,Middle Eastern Restaurant,Bank
2,Downtown Toronto,0.0,Coffee Shop,Italian Restaurant,Cocktail Bar,Restaurant,Gastropub,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Moroccan Restaurant
3,East Toronto,0.0,Health Food Store,Playground,Pub,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
4,Downtown Toronto,0.0,Cocktail Bar,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Butcher,Restaurant
5,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Sandwich Place,Pizza Place,Sushi Restaurant,Plaza,Middle Eastern Restaurant,Japanese Restaurant,Bank,Cosmetics Shop
6,Downtown Toronto,0.0,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
7,Downtown Toronto,0.0,Coffee Shop,Café,Sandwich Place,Hotel,Asian Restaurant,Japanese Restaurant,Restaurant,Bank,Gym,Deli / Bodega
8,West Toronto,0.0,Park,Pharmacy,Pizza Place,Bus Line,Liquor Store,Middle Eastern Restaurant,Furniture / Home Store,Bank,Bakery,Bar
10,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,Japanese Restaurant,Hotel,Boat or Ferry,Park,Aquarium,Bank,Restaurant,Spa


## Cluster 2 - Convenience Store Route

In [199]:
df.loc[df['Cluster Labels'] == 1, df.columns[[1] + list(range(5, df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East York/East ...,1.0,Convenience Store,Park,Intersection,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
22,West Toronto,1.0,Convenience Store,Park,Residential Building (Apartment / Condo),Yoga Studio,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
29,Central Toronto,1.0,Convenience Store,Park,Gym,Restaurant,Yoga Studio,Distribution Center,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


## Cluster 3 - Park Route

In [200]:
df.loc[df['Cluster Labels'] == 2, df.columns[[1] + list(range(5, df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2.0,Park,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


## Cluster 4 - WorkOut Route

In [201]:
df.loc[df['Cluster Labels'] == 3, df.columns[[1] + list(range(5, df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3.0,Gym Pool,Playground,Park,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
33,Downtown Toronto,3.0,Tennis Court,Park,Gym / Fitness Center,Playground,Discount Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


## Cluster 5 - Miscelaneous Route

In [202]:
df.loc[df['Cluster Labels'] == 4, df.columns[[1] + list(range(5, df.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Central Toronto,4.0,Coffee Shop,Light Rail Station,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
